#### 仿射变换

In [ ]:
# 平移
import cv2
import numpy as np
import matplotlib.pyplot as plt
img = cv2.imread('../../image/1.jpg')
img= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
height,width = img.shape
# 平移distancd
tx = 100
ty = 100
# 定义转换矩阵M
M = np.float32([[1,0,tx],[0,1,ty]])
# 对原始图像执行变换矩阵M的转换，大小保持不变，输出图像dst
dst = cv2.warpAffine(img,M,(width,height))
plt.imshow(dst,'gray')
plt.show()

In [ ]:
# 旋转
center = (width/2,height/2)
# 旋转角度
angle = 45
#缩放因子 缩小一倍
scale=0.5
# 通过cv2.getRotationMatrix2D获取矩阵
M = cv2.getRotationMatrix2D(center,angle,scale)
# 通过cv2.warpAffine函数变换矩阵应用到img上
dst = cv2.warpAffine(img,M,(width,height))
plt.imshow(dst,'gray')
plt.show()

#### 透视变换

In [ ]:
# 将图片正视化
# 预处理
dst.shape
gray = cv2.resize(dst,(400,600))

# 高斯滤波
Guass = cv2.GaussianBlur(gray,(5,5),0)
# canny边缘检测
Canny = cv2.Canny(Guass.copy(),100,200)
# 图像轮廓   线
contours = cv2.findContours(Canny.copy(),cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)[0]
# 找最大的轮廓
max_list_contours = sorted(contours,key=cv2.contourArea,reverse=True)[:5]
# 筛选并进行轮廓近似，画矩形
screenCnt=0
for cnt in max_list_contours:
    epsilon = 0.02 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    if len(approx) == 4:
        screenCnt = approx
        print(1)
        break



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
img = cv2.imread('../../image/1.jpg')
img= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img = cv2.resize(img,(400,600))
# 变换之前的坐标位置
rect= np.array([[0,0],[400,0],[400,600],[0,600]],dtype='float32')
# 变换之后的坐标位置
dst = np.array([[100,0],[300,0],[300,500],[0,500]],dtype='float32')
M = cv2.getPerspectiveTransform(rect, dst)
warped = cv2.warpPerspective(img.copy(), M, (300, 500))
'''
 src:输入图像矩阵
 M3*3的透视变换矩阵,可以通过getPerspectiveTransform等函数获取
 dsize:结果图像大小，为宽和高的二元组
 dst:输出结果图像，可以省略，结果图像会作为函数处理结果输出
 flags:可选参数,插值方法的组合(int 类型），默认值 INTER_LINEAR,
 本函数官方材料说明取值为INTER_LINEAR 或 INTER_NEAREST与 WARP_INVERSE_MAP的组合,
 但老猿测试其他标志也是支持的,具体取值及含义请参考
'''
cv2.imshow('Scanned', warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

#### 特征匹配


In [21]:
# 带有SIFT描述符和比例测试的BF匹配：
import cv2
img1 = cv2.imread('../../image/1.jpg')
# 设一个ROI为被匹配的图像
ROI = img1[400:800,300:800]            # trainImage
img1=cv2.resize(img,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_CUBIC)
img2=cv2.resize(ROI,(img1.shape[1],img1.shape[0]))
# 初始化SIFT特征点检测器
sift = cv2.xfeatures2d.SIFT_create()
# 检测特征点与描述符
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
# 创建蛮力（BF）匹配器
bf = cv2.BFMatcher()
matches = bf.knnMatch(des1,des2, k=2)
# 比值测试，首先获取与 A 距离最近的点 B（最近）和 C（次近），只有当 B/C
# 小于阈值时（ 0.75）才被认为是匹配，因为假设匹配是一一对应的，真正的匹配的理想距离为 0
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])
# cv.drawMatchesKnn()把列表作为匹配项。
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good[:10],None,flags=2)
cv2.imshow("show",img3)
cv2.waitKey()
cv2.destroyAllWindows()